In [0]:
dbutils.widgets.text("catalog_name","")
dbutils.widgets.text("schema_name","")
dbutils.widgets.text("warehouse_id","")

catalog_name=dbutils.widgets.get("catalog_name")
schema_name=dbutils.widgets.get("schema_name")
warehouse_id=dbutils.widgets.get("warehouse_id")

In [0]:
from databricks.sdk.service.catalog import MonitorInfo, MonitorNotifications, MonitorSnapshot,MonitorCronSchedule
from databricks.sdk import WorkspaceClient

In [0]:
%py
sql(f"use catalog {catalog_name}");
sql(f"use schema {schema_name}");

In [0]:
%py
df=sql("show tables")
tableName=[i.tableName for i in df.collect() if(i.tableName!='_sqldf')]

In [0]:
def f_create_quality_monitor(catalog_name,schema_name,table_name,assets_dir,warehouse_id,workspaceclient:WorkspaceClient):

    snapshot_config = MonitorSnapshot()
    monitor_notifications = None
    schedule = MonitorCronSchedule(quartz_cron_expression='0 4 19 * * ?', timezone_id='UTC')
    monitor_info = workspaceclient.quality_monitors.create(
                table_name=f'{catalog_name}.{schema_name}.{table_name}',
                assets_dir=assets_dir,
                output_schema_name=f"{catalog_name}.silver",
                notifications=monitor_notifications,
                schedule=schedule,
                snapshot=snapshot_config,
                warehouse_id=warehouse_id
            )

In [0]:
w=WorkspaceClient()
for i in tableName:
    f_create_quality_monitor(catalog_name,schema_name,i,"/GeekCoders_LLM/GeekCoders_LLM/databricks_lakehouse_monitoring", warehouse_id,w)